In [1]:
import pandas as pd
import numpy as np
from scipy import ndimage #ndimage provides a large amount of functions for image processing: https://docs.scipy.org/doc/scipy/reference/ndimage.html 
from skimage import color #many tutorials: https://github.com/scikit-image/skimage-tutorials/tree/master/lectures
import matplotlib.pyplot as plt # matplotlib is the major lib for plotting
import imageio
from PIL import Image
import os

In [2]:
PATH_TO_IMAGE_DIR = 'VALIDATION_DATA'

In [3]:
PATH_OUT = 'VALIDATION_DATA_CLEANED'

In [4]:
os.listdir(PATH_TO_IMAGE_DIR)

['image_table.csv',
 '.DS_Store',
 '04_HOUSEHOLD',
 'labels_concatenated.csv',
 '02_GROCERY',
 '01_AUTOMOBIL',
 '03_REALESTATE']

In [6]:
#loop throug the single datasource directories
for directory in os.listdir(PATH_TO_IMAGE_DIR):
    if directory == '.DS_Store':
        continue
    else:
        dir = str.split(directory, '/')[-1]
        df = pd.read_csv(os.path.join(PATH_TO_IMAGE_DIR, directory, 'labels.csv'))
        print('df read')
        #print(dir)
        i = 0
        #loop throug the single images
        for filename in os.listdir(os.path.join(PATH_TO_IMAGE_DIR, directory)):
            #check file ending
            if filename.endswith('.csv'):
                continue
            if filename.endswith('.jpg') or filename.endswith('.png') or filename.endswith('.png') or filename.endswith('.jpeg'):
                try:
                    img = Image.open(os.path.join(PATH_TO_IMAGE_DIR, directory, filename)) # open the image file
                    img.verify() # verify that it is, in fact an image
                    #rename image
                    f_nam = dir + '_image' + '_' + str(i) + ".jpg"
                    print(filename, '->', f_nam)
                    if directory == '03_REALESTATE':
                        df.replace('scaled_' + filename, f_nam, inplace = True)
                        i += 1
                        os.rename(PATH_TO_IMAGE_DIR + '/' + directory + '/' + filename, PATH_TO_IMAGE_DIR + '/' + directory + '/' + f_nam)
                    else:
                        df.replace(filename, f_nam, inplace = True)
                        i += 1
                        os.rename(PATH_TO_IMAGE_DIR + '/' + directory + '/' + filename, PATH_TO_IMAGE_DIR + '/' + directory + '/' + f_nam)
                except (IOError, SyntaxError) as e:
                    print('Bad file:', filename)
                    #os.remove(os.path.join(PATH_TO_IMAGE_DIR, directory , filename)) 
        df.to_csv(os.path.join(PATH_OUT, directory + '.csv'))
        
            
    

df read
04_HOUSEHOLD_image_102.jpg -> 04_HOUSEHOLD_image_0.jpg
04_HOUSEHOLD_image_116.jpg -> 04_HOUSEHOLD_image_1.jpg
04_HOUSEHOLD_image_85.jpg -> 04_HOUSEHOLD_image_2.jpg
04_HOUSEHOLD_image_91.jpg -> 04_HOUSEHOLD_image_3.jpg
04_HOUSEHOLD_image_46.jpg -> 04_HOUSEHOLD_image_4.jpg
04_HOUSEHOLD_image_52.jpg -> 04_HOUSEHOLD_image_5.jpg
04_HOUSEHOLD_image_53.jpg -> 04_HOUSEHOLD_image_6.jpg
04_HOUSEHOLD_image_47.jpg -> 04_HOUSEHOLD_image_7.jpg
04_HOUSEHOLD_image_90.jpg -> 04_HOUSEHOLD_image_8.jpg
04_HOUSEHOLD_image_84.jpg -> 04_HOUSEHOLD_image_9.jpg
04_HOUSEHOLD_image_117.jpg -> 04_HOUSEHOLD_image_10.jpg
04_HOUSEHOLD_image_103.jpg -> 04_HOUSEHOLD_image_11.jpg
04_HOUSEHOLD_image_129.jpg -> 04_HOUSEHOLD_image_12.jpg
04_HOUSEHOLD_image_115.jpg -> 04_HOUSEHOLD_image_13.jpg
04_HOUSEHOLD_image_101.jpg -> 04_HOUSEHOLD_image_14.jpg
04_HOUSEHOLD_image_92.jpg -> 04_HOUSEHOLD_image_15.jpg
04_HOUSEHOLD_image_86.jpg -> 04_HOUSEHOLD_image_16.jpg
04_HOUSEHOLD_image_51.jpg -> 04_HOUSEHOLD_image_17.jpg
04_HO

In [7]:

df = pd.DataFrame()
for filename in os.listdir(PATH_OUT):
        df_1 = pd.read_csv(os.path.join(PATH_OUT, filename))
        frame = [df, df_1]
        df = pd.concat(frame)
df.to_csv(PATH_TO_IMAGE_DIR + '/labels_concatenated.csv')

In [13]:
#get shapes of images into dataframe
df_images = pd.DataFrame()
for directory in os.listdir(PATH_TO_IMAGE_DIR):
    if directory == '.DS_Store' or directory == 'labels_concatenated.csv':
        continue
    else:
        im_y = []
        im_x = []
        im_filename = []
        for filename in os.listdir(os.path.join(PATH_TO_IMAGE_DIR, directory)):
            #read image
            if filename == '.DS_Store' or filename == 'labels.csv':
                continue
            else:
                try:
                    im = imageio.imread(os.path.join(PATH_TO_IMAGE_DIR, directory, filename))
                    im_y.append(im.shape[0])
                    im_x.append(im.shape[1])
                    im_filename.append(str.split(filename, '.')[0])
                except:
                    print(f'{filename} failed')
                
        im_dic = {'Width':im_x, 'Height': im_y, 'Filename': im_filename, 'Folder': directory}
        df_im = pd.DataFrame(im_dic)
        df_im['aspect_ratio'] = df_im['Height']/df_im['Width']
        df_im['aspect_ratio_round'] = round(df_im['aspect_ratio'], 1)
        f = [df_im, df_images]
        df_images = pd.concat(f)

/Users/lorenzogiurintano/opt/anaconda3/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:822: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 2. 
  warnings.warn(str(msg))


In [15]:
df_images.to_csv(PATH_TO_IMAGE_DIR + '/image_table.csv')